In [1]:
import numpy as np
import pandas as pd

import featuretools as ft
from dask import bag
from dask.diagnostics import ProgressBar
import pandas as pd
import utils

In [2]:
file_Path_Event_Report_Ahmedabad = r'Data/Events Report of Drivers in Ahmedabad_16-07 to 31-07.xlsx'

file_Path_Bangalore_CAS_Alerts = r'Data/bangalore-cas-alerts.csv'


data_Ahmedabad_New_Column_Name = ['Event_Name', 'Event_Time', 'Event_Value','Event_Unit', 'Bus_N0.', 'Route_Name',
                                  'Stop_Code', 'Stop_Name', 'Latitude', 'Longitude']
data_Banglore_New_Column_Name = ['Device_Code', 'Latitude', 'Longitude', 'Ward_Name', 'Alarm_Type', 'Speed', 'Date_Time']

In [3]:
# data_Ahmedabad = pd.read_excel(file_Path_Event_Report_Ahmedabad)
# data_Ahmedabad.columns = data_Ahmedabad_New_Column_Name

In [4]:
data_Banglore = pd.read_csv(file_Path_Bangalore_CAS_Alerts)
data_Banglore.columns = data_Banglore_New_Column_Name

In [5]:
# data_Ahmedabad['Event_Time'] = pd.to_datetime(data_Ahmedabad['Event_Time'])

data_Banglore['Date_Time'] = pd.to_datetime(data_Banglore['Date_Time'])

data_Banglore.drop_duplicates(inplace = True)

data_Banglore.reset_index(inplace=True, drop=True)

# data_Banglore

In [6]:
len(data_Banglore['Date_Time'].unique())

134441

In [7]:
# data_Banglore.to_csv('Banglore_Data.csv', index = False)

In [8]:
def Clean_Banglore_Data(data_Banglore):
    
    rows_To_Delete = []
    
    for index in range(len(data_Banglore)):
                
        if data_Banglore['Ward_Name'][index] == 'other':
            
            previous_Device_Code = data_Banglore['Device_Code'][index - 1]
            previous_Latitude = data_Banglore['Latitude'][index - 1]
            previous_Longitude = data_Banglore['Longitude'][index - 1]
            previous_Alarm_Type = data_Banglore['Alarm_Type'][index - 1]
            previous_Speed = data_Banglore['Speed'][index - 1]
            previous_Date_Time = data_Banglore['Date_Time'][index - 1]
            
            current_Device_Code = data_Banglore['Device_Code'][index]
            current_Latitude = data_Banglore['Latitude'][index]
            current_Longitude = data_Banglore['Longitude'][index]
            current_Alarm_Type = data_Banglore['Alarm_Type'][index]
            current_Speed = data_Banglore['Speed'][index]
            current_Date_Time = data_Banglore['Date_Time'][index]
            
            if( (previous_Device_Code == current_Device_Code) and (previous_Latitude == current_Latitude) and
                (previous_Longitude == current_Longitude) and (previous_Alarm_Type == current_Alarm_Type) and 
                (previous_Speed == current_Speed) and (previous_Date_Time == current_Date_Time)):
                
                rows_To_Delete.append(index)
                
    print(len(rows_To_Delete))
    
    return rows_To_Delete

rows_To_Delete = Clean_Banglore_Data(data_Banglore)    

4455


In [9]:
print(len(rows_To_Delete))
data_Banglore.drop(data_Banglore.index[rows_To_Delete], inplace=True)
data_Banglore.reset_index(inplace=True, drop=True)

print(len(data_Banglore))

# rows_To_Delete = Clean_Banglore_Data(data_Banglore)   
# print(len(rows_To_Delete))
# data_Banglore.drop(data_Banglore.index[rows_To_Delete], inplace=True)


4455
147821


In [10]:
# for index in range(len(data_Banglore)):
                
#         if data_Banglore['Ward_Name'][index] == 'other':
            
#             if index not in rows_To_Delete:
#                 print (index)

print(len(data_Banglore) - len(rows_To_Delete))

data_temp = data_Banglore.copy()

data_temp.drop(labels=['Ward_Name'], axis = 1, inplace=True)
data_temp.reset_index(inplace=True, drop=True)
# print(data_temp)
# data_temp.drop(labels=['index','level_0'], axis = 1, inplace=True)


data_temp.drop_duplicates(inplace = True)
# print(data_temp)
print(len(data_temp))

data_Banglore.to_csv('Banglore_Data.csv', index = False)

143366
146944


In [11]:
# data_Ahmedabad.head(10)

In [12]:
# e = pd.Series(range(0, len(data_Ahmedabad)))

In [13]:
# data_Ahmedabad = data_Ahmedabad.assign(Index_Col = e.values)
# data_Banglore

In [14]:
# df_Warn.sort_values(by = ['Device_Code', 'Alarm_Type'], inplace = True)
# df_Warn
# df_Warn['Count'][1]

In [15]:
df_Warn_Agg = pd.DataFrame(columns = ['Alarm_Type', 'Count', 'Percentage_Count'])

total_Warning = len(data_Banglore)

for warn_type in data_Banglore.Alarm_Type.unique():
    df = data_Banglore.loc[data_Banglore['Alarm_Type'] == warn_type]
    
    interim_Count = len(df)
    
    df_Warn_Agg.loc[len(df_Warn_Agg)] = [warn_type, interim_Count, ((interim_Count/total_Warning)*100)]

In [16]:
df_Warn_Agg.to_csv('Banglore_Alarm_Type_Agg.csv', index = False)

df_Warn_Agg

,Alarm_Type,Count,Percentage_Count
0,PCW,17122,11.582928
1,FCW,25092,16.974584
2,Overspeed,19245,13.019124
3,HMW,25416,17.193768
4,UFCW,59083,39.969287
5,LDWL,1008,0.681906
6,LDWR,855,0.578402


In [17]:
warn_column = ['Device_Code', 'Alarm_Type', 'Count', 'Device_Percentage_Count', 'Alarm_Type_Percentage_Count', 'Overall_Percentage_Count']
df_Warn = pd.DataFrame(columns = warn_column)

for device_Code in data_Banglore.Device_Code.unique():
    df = data_Banglore.loc[data_Banglore['Device_Code'] == device_Code]
    
    for alarm_Type in df.Alarm_Type.unique():
        df_New = df.loc[df['Alarm_Type'] == alarm_Type]
        
        alarm_Type_Percentage_Count = df_Warn_Agg.loc[df_Warn_Agg['Alarm_Type'] == alarm_Type, 'Count'].iloc[0]
#         print(alarm_Type_Percentage_Count)
        alarm_Type_Percentage_Count = (len(df_New)/alarm_Type_Percentage_Count)*100
        overall_Percentage_Count = (len(df_New)/len(data_Banglore)) * 100
        
        df_Warn.loc[len(df_Warn)] = [device_Code, alarm_Type, len(df_New), (len(df_New)/len(df))*100, alarm_Type_Percentage_Count,
                                    overall_Percentage_Count]

In [18]:
df_Warn.to_csv('Banglore_Alarm_By_Device_Type.csv', index = False)

In [19]:
df_Warn

,Device_Code,Alarm_Type,Count,Device_Percentage_Count,Alarm_Type_Percentage_Count,Overall_Percentage_Count
0,864504031502210,PCW,444,8.922830,2.593155,0.300363
1,864504031502210,FCW,381,7.656752,1.518412,0.257744
2,864504031502210,Overspeed,2086,41.921222,10.839179,1.411166
3,864504031502210,HMW,657,13.203376,2.584986,0.444456
4,864504031502210,UFCW,1406,28.255627,2.379703,0.951150
5,864504031502210,LDWL,2,0.040193,0.198413,0.001353
6,863977033764239,PCW,842,7.892763,4.917650,0.569608
7,863977033764239,UFCW,2978,27.915261,5.040367,2.014599
8,863977033764239,FCW,2249,21.081740,8.963016,1.521435
9,863977033764239,HMW,3076,28.833896,12.102613,2.080895


In [20]:
columns = ['Ward_Name', 'Count', 'Percentage_Count']

df_Ward_Name_Agg = pd.DataFrame(columns=columns )

total_Warning = len(data_Banglore)

for ward_Name in data_Banglore.Ward_Name.unique():
    df = data_Banglore.loc[data_Banglore['Ward_Name'] == ward_Name]
    
    interim_Count = len(df)
    
    df_Ward_Name_Agg.loc[len(df_Ward_Name_Agg)] = [ward_Name, interim_Count, ((interim_Count/total_Warning)*100)]
    
    

In [21]:
df_Ward_Name_Agg.sort_values(by = ['Percentage_Count'], inplace = True)

df_Ward_Name_Agg.reset_index(inplace=True, drop=True)

df_Ward_Name_Agg.to_csv('Banglore_Ward_Type_Agg.csv', index=False)

df_Ward_Name_Agg

,Ward_Name,Count,Percentage_Count
0,Yelchenahalli,1,0.000676
1,Sudham Nagara,1,0.000676
2,Jayanagar East,3,0.002029
3,Kammanahalli,3,0.002029
4,Gurappanapalya,4,0.002706
5,Jaraganahalli,5,0.003382
6,Bharathi Nagar,5,0.003382
7,Dharmaraya Swamy Temple,7,0.004735
8,Chickpete,9,0.006088
9,J P Nagar,11,0.007441


In [22]:
ward_column = ['Ward_Name', 'Alarm_Type', 'Count', 'Ward_Percentage_Count', 'Alarm_Type_Percentage_Count', 'Overall_Percentage_Count']
df_Ward = pd.DataFrame(columns = ward_column)

for ward_Name in data_Banglore.Ward_Name.unique():
    df = data_Banglore.loc[data_Banglore['Ward_Name'] == ward_Name]
#     print(ward_Name)
    
    for alarm_Type in df.Alarm_Type.unique():
        df_New = df.loc[df['Alarm_Type'] == alarm_Type]
        
        alarm_Type_Percentage_Count = df_Warn_Agg.loc[df_Warn_Agg['Alarm_Type'] == alarm_Type, 'Count'].iloc[0]
#         print(alarm_Type_Percentage_Count)
        alarm_Type_Percentage_Count = (len(df_New)/alarm_Type_Percentage_Count)*100
        
        overall_Percentage_Count = (len(df_New)/len(data_Banglore)) * 100

        
        df_Ward.loc[len(df_Ward)] = [ward_Name, alarm_Type, len(df_New), (len(df_New)/len(df))*100, alarm_Type_Percentage_Count,
                                     overall_Percentage_Count]

In [23]:
df_Ward

,Ward_Name,Alarm_Type,Count,Ward_Percentage_Count,Alarm_Type_Percentage_Count,Overall_Percentage_Count
0,Kadugodi,PCW,2297,14.869239,13.415489,1.553906
1,Kadugodi,HMW,2059,13.328586,8.101196,1.392901
2,Kadugodi,UFCW,6071,39.299586,10.275375,4.106994
3,Kadugodi,Overspeed,2369,15.335318,12.309691,1.602614
4,Kadugodi,FCW,2541,16.448731,10.126734,1.718971
5,Kadugodi,LDWR,79,0.511393,9.239766,0.053443
6,Kadugodi,LDWL,32,0.207147,3.174603,0.021648
7,Garudachar Playa,FCW,1046,17.831572,4.168659,0.707613
8,Garudachar Playa,UFCW,1834,31.264916,3.104108,1.240690
9,Garudachar Playa,HMW,965,16.450733,3.796821,0.652817


In [24]:
df_Ward.to_csv('Banglore_Alarm_By_Ward_Type_.csv', index=False)